# 08 迁移学习

> 站在巨人的肩膀上：使用预训练模型

---

## 📚 学习目标

- [ ] 理解迁移学习的原理
- [ ] 使用 PyTorch 预训练模型
- [ ] 掌握特征提取和微调两种方法

In [1]:
import numpy as np
import matplotlib.pyplot as plt

try:
    import torch
    import torch.nn as nn
    import torchvision.models as models
    import torchvision.transforms as transforms
    HAS_TORCH = True
except ImportError:
    HAS_TORCH = False
    print("请安装 PyTorch 和 torchvision")

---

## 💡 为什么迁移学习有效？

### CNN 的特征层次

```
浅层（靠近输入）      深层（靠近输出）
     ↓                    ↓
  边缘、颜色    →    纹理、形状    →    高级语义
     ↓                    ↓                ↓
  通用特征            领域特定         任务特定
```

**关键洞察**：浅层特征（边缘、纹理）在不同任务中是通用的！

### 迁移学习策略

1. **特征提取 (Feature Extraction)**：冻结预训练层，只训练新的分类头
2. **微调 (Fine-tuning)**：解冻部分/全部层，用小学习率继续训练

---

## 💻 加载预训练模型

In [2]:
if HAS_TORCH:
    # 加载预训练的 ResNet-18
    # weights='DEFAULT' 使用 ImageNet 预训练权重
    resnet = models.resnet18(weights='DEFAULT')
    
    print("ResNet-18 结构（最后几层）:")
    print("...")
    for name, module in list(resnet.named_children())[-3:]:
        print(f"{name}: {module}")
    
    print(f"\n原始分类头: {resnet.fc.in_features} → {resnet.fc.out_features} (ImageNet 1000类)")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/lyh/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44.7M/44.7M [01:30<00:00, 519kB/s]


ResNet-18 结构（最后几层）:
...
layer4: Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

---

## 💻 方法1：特征提取

In [3]:
if HAS_TORCH:
    # 复制模型
    model_feature_extract = models.resnet18(weights='DEFAULT')
    
    # 冻结所有层
    for param in model_feature_extract.parameters():
        param.requires_grad = False
    
    # 替换分类头（假设新任务是 5 分类）
    num_classes = 5
    num_features = model_feature_extract.fc.in_features
    model_feature_extract.fc = nn.Linear(num_features, num_classes)
    
    # 统计可训练参数
    trainable_params = sum(p.numel() for p in model_feature_extract.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model_feature_extract.parameters())
    
    print("特征提取模式:")
    print(f"  总参数: {total_params:,}")
    print(f"  可训练参数: {trainable_params:,}")
    print(f"  冻结比例: {(1 - trainable_params/total_params)*100:.1f}%")
    print(f"\n💡 只需要训练最后一层！")

特征提取模式:
  总参数: 11,179,077
  可训练参数: 2,565
  冻结比例: 100.0%

💡 只需要训练最后一层！


---

## 💻 方法2：微调

In [ ]:
if HAS_TORCH:
    # 复制模型
    model_finetune = models.resnet18(weights='DEFAULT')
    
    # 冻结前面的层（layer1, layer2）
    for name, param in model_finetune.named_parameters():
        if 'layer3' not in name and 'layer4' not in name and 'fc' not in name:
            param.requires_grad = False
    
    # 替换分类头
    model_finetune.fc = nn.Linear(model_finetune.fc.in_features, num_classes)
    
    # 统计可训练参数
    trainable_params = sum(p.numel() for p in model_finetune.parameters() if p.requires_grad)
    
    print("微调模式 (解冻 layer3, layer4, fc):")
    print(f"  可训练参数: {trainable_params:,}")
    
    # 列出可训练的层
    print("\n可训练的层:")
    for name, param in model_finetune.named_parameters():
        if param.requires_grad:
            print(f"  {name}")

---

# Part 2: 完整迁移学习项目

> 使用 **Oxford Flowers 102** 数据集，对比三种策略的实际效果

## 2.1 数据准备

In [ ]:
# ============================================================
# 2.1 加载 Flowers-102 数据集
# ============================================================

from torchvision import datasets
from torch.utils.data import DataLoader, Subset

# 数据增强
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 下载数据集
print("下载 Flowers-102 数据集...")
train_dataset = datasets.Flowers102(root='./data', split='train', transform=train_transform, download=True)
val_dataset = datasets.Flowers102(root='./data', split='val', transform=test_transform, download=True)

print(f"训练集: {len(train_dataset)} 张, 验证集: {len(val_dataset)} 张, 类别数: 102")

# 创建 DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

---

## 2.2 定义三种策略的模型

我们将对比三种迁移学习策略：
1. **从零训练 (Scratch)**：随机初始化，训练所有层
2. **特征提取 (Feature Extraction)**：冻结预训练层，只训练分类头
3. **微调 (Fine-tuning)**：使用预训练权重，解冻部分层继续训练

In [ ]:
# ============================================================
# 定义三种策略的模型
# ============================================================

num_classes = 102

def create_model_scratch():
    model = models.resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

def create_model_feature_extract():
    model = models.resnet18(weights='DEFAULT')
    for param in model.parameters():
        param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

def create_model_finetune():
    model = models.resnet18(weights='DEFAULT')
    for name, param in model.named_parameters():
        if 'layer4' not in name and 'fc' not in name:
            param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# 参数统计
print("参数统计：")
for name, fn in [('Scratch', create_model_scratch), ('Feature Extract', create_model_feature_extract), ('Fine-tune', create_model_finetune)]:
    m = fn()
    trainable = sum(p.numel() for p in m.parameters() if p.requires_grad)
    print(f"  {name}: {trainable:,} 可训练参数")

---

## 2.3 训练与对比

In [ ]:
# ============================================================
# 训练函数
# ============================================================

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    correct, total = 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return 100. * correct / total

def evaluate(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100. * correct / total

def train_model(model, train_loader, val_loader, epochs, lr, device, name):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    
    history = {'train_acc': [], 'val_acc': []}
    print(f"\n训练: {name}")
    
    for epoch in range(epochs):
        train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_acc = evaluate(model, val_loader, device)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        print(f"  Epoch {epoch+1}: Train={train_acc:.1f}%, Val={val_acc:.1f}%")
    
    return history

In [ ]:
# ============================================================
# 训练三种策略并对比
# ============================================================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

EPOCHS = 5  # 演示用，实际可增加到 20-50
LR = 1e-3

all_histories = {}

# 1. 从零训练
model1 = create_model_scratch()
h1 = train_model(model1, train_loader, val_loader, EPOCHS, LR, device, "从零训练 (Scratch)")
all_histories['Scratch'] = h1

# 2. 特征提取
model2 = create_model_feature_extract()
h2 = train_model(model2, train_loader, val_loader, EPOCHS, LR, device, "特征提取 (Feature Extract)")
all_histories['Feature Extract'] = h2

# 3. 微调
model3 = create_model_finetune()
h3 = train_model(model3, train_loader, val_loader, EPOCHS, LR*0.1, device, "微调 (Fine-tune)")
all_histories['Fine-tune'] = h3

print("\n✅ 训练完成！")

In [ ]:
# ============================================================
# 可视化对比结果
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = {'Scratch': 'red', 'Feature Extract': 'blue', 'Fine-tune': 'green'}
epochs_range = range(1, EPOCHS + 1)

# 训练曲线
ax = axes[0]
for name, history in all_histories.items():
    ax.plot(epochs_range, history['train_acc'], color=colors[name], 
            linestyle='-', linewidth=2, label=f'{name} (Train)')
    ax.plot(epochs_range, history['val_acc'], color=colors[name],
            linestyle='--', linewidth=2, label=f'{name} (Val)')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy (%)')
ax.set_title('训练曲线对比')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

# 最终准确率柱状图
ax = axes[1]
names = list(all_histories.keys())
final_val_acc = [all_histories[n]['val_acc'][-1] for n in names]

bars = ax.bar(names, final_val_acc, color=[colors[n] for n in names])
ax.set_ylabel('Validation Accuracy (%)')
ax.set_title('最终验证准确率对比')

for bar, acc in zip(bars, final_val_acc):
    ax.annotate(f'{acc:.1f}%', xy=(bar.get_x() + bar.get_width()/2, acc),
                xytext=(0, 3), textcoords="offset points", ha='center')

plt.tight_layout()
plt.show()

# 打印总结
print("\n" + "="*50)
print("结果总结")
print("="*50)
for name in names:
    print(f"  {name:<20}: {all_histories[name]['val_acc'][-1]:.1f}%")

print("\n💡 关键发现：")
print("   - 迁移学习比从零训练收敛更快")
print("   - 预训练权重提供了很好的初始化")

---

# Part 3: 进阶话题

## 3.1 域差异 (Domain Shift)

当源域（预训练数据）和目标域（你的数据）差异较大时，迁移效果会下降。

```
域差异示例：
┌─────────────────────────────────────────────────────────────┐
│  小域差异                          大域差异                │
│  ─────────                        ─────────                │
│  ImageNet → 花卉分类              ImageNet → 医学影像      │
│  ImageNet → 宠物分类              ImageNet → 卫星图像      │
│  ImageNet → 食物分类              ImageNet → 显微镜图像    │
│                                                             │
│  策略：冻结更多层                  策略：解冻更多层         │
└─────────────────────────────────────────────────────────────┘
```

## 3.2 负迁移 (Negative Transfer)

**负迁移**：使用预训练权重反而比从零训练效果更差。

**常见原因**：
1. 源域和目标域差异太大
2. 预训练模型过于复杂（目标数据太少）
3. 学习率设置不当

**如何避免**：
- 先做小规模实验对比
- 从冻结更多层开始，逐步解冻
- 监控验证集表现

## 3.3 分层学习率

**核心思想**：不同层使用不同的学习率
- 浅层（通用特征）：小学习率
- 深层（任务特征）：大学习率

---

## 3.4 不同预训练模型对比

ResNet 家族有多个版本，参数量和性能不同：

In [6]:
# ============================================================
# 不同 ResNet 版本对比
# ============================================================

# ResNet 家族
resnet_models = {
    'ResNet-18': models.resnet18,
    'ResNet-34': models.resnet34,
    'ResNet-50': models.resnet50,
    'ResNet-101': models.resnet101,
    'ResNet-152': models.resnet152,
}

print("ResNet 家族参数量对比：")
print("=" * 60)
print(f"{'模型':<15} {'参数量':>15} {'ImageNet Top-1':>15}")
print("-" * 60)

# ImageNet 准确率数据（来自 PyTorch 官方）
imagenet_acc = {
    'ResNet-18': 69.8,
    'ResNet-34': 73.3,
    'ResNet-50': 76.1,
    'ResNet-101': 77.4,
    'ResNet-152': 78.3,
}

for name, model_fn in resnet_models.items():
    model = model_fn(weights=None)  # 不下载权重，只看结构
    params = sum(p.numel() for p in model.parameters())
    acc = imagenet_acc[name]
    print(f"{name:<15} {params/1e6:>12.1f}M {acc:>14.1f}%")

print("\n💡 选择建议：")
print("   - 小数据集/快速实验: ResNet-18 (参数少，训练快)")
print("   - 追求精度: ResNet-50 (性价比最高)")
print("   - 极致精度: ResNet-101/152 (需要更多数据和算力)")

ResNet 家族参数量对比：
模型                          参数量  ImageNet Top-1
------------------------------------------------------------
ResNet-18               11.7M           69.8%
ResNet-34               21.8M           73.3%
ResNet-50               25.6M           76.1%
ResNet-101              44.5M           77.4%
ResNet-152              60.2M           78.3%

💡 选择建议：
   - 小数据集/快速实验: ResNet-18 (参数少，训练快)
   - 追求精度: ResNet-50 (性价比最高)
   - 极致精度: ResNet-101/152 (需要更多数据和算力)


In [7]:
# ============================================================
# 使用不同 ResNet 做迁移学习
# ============================================================

def create_transfer_model(model_name='resnet18', num_classes=102, mode='finetune'):
    """
    创建迁移学习模型
    
    参数:
        model_name: 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152'
        num_classes: 目标类别数
        mode: 'feature_extract' 或 'finetune'
    """
    # 加载预训练模型
    model_fn = getattr(models, model_name)
    model = model_fn(weights='DEFAULT')
    
    # 获取分类头的输入特征数
    # ResNet-18/34: fc.in_features = 512
    # ResNet-50/101/152: fc.in_features = 2048
    in_features = model.fc.in_features
    
    if mode == 'feature_extract':
        # 冻结所有层
        for param in model.parameters():
            param.requires_grad = False
    
    elif mode == 'finetune':
        # 冻结早期层，解冻后期层
        for name, param in model.named_parameters():
            if 'layer4' not in name and 'fc' not in name:
                param.requires_grad = False
    
    # 替换分类头
    model.fc = nn.Linear(in_features, num_classes)
    
    return model

# 演示：创建不同版本的迁移学习模型
print("不同 ResNet 做迁移学习（102 类花卉）：")
print("=" * 60)

for model_name in ['resnet18', 'resnet50']:
    model = create_transfer_model(model_name, num_classes=102, mode='finetune')
    
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"\n{model_name.upper()}:")
    print(f"  分类头输入: {model.fc.in_features} 维")
    print(f"  总参数: {total/1e6:.1f}M")
    print(f"  可训练参数: {trainable/1e6:.1f}M ({trainable/total*100:.1f}%)")

不同 ResNet 做迁移学习（102 类花卉）：

RESNET18:
  分类头输入: 512 维
  总参数: 11.2M
  可训练参数: 8.4M (75.2%)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /Users/lyh/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [01:01<00:00, 1.66MB/s]



RESNET50:
  分类头输入: 2048 维
  总参数: 23.7M
  可训练参数: 15.2M (64.0%)


### ResNet-18 vs ResNet-50 关键区别

```
ResNet-18/34: 使用 BasicBlock (2层卷积)
┌────────────────────┐
│ Conv 3×3 → BN → ReLU │
│ Conv 3×3 → BN       │
│        + x          │
└────────────────────┘
fc.in_features = 512

ResNet-50/101/152: 使用 Bottleneck (3层卷积)
┌────────────────────┐
│ Conv 1×1 → BN → ReLU │  ← 降维
│ Conv 3×3 → BN → ReLU │  ← 提特征
│ Conv 1×1 → BN       │  ← 升维
│        + x          │
└────────────────────┘
fc.in_features = 2048
```

**实际使用建议**：
- 快速验证想法 → ResNet-18
- 正式项目 → ResNet-50（性价比最高）
- 计算资源充足 → ResNet-101

In [ ]:
# ============================================================
# 分层学习率示例
# ============================================================

def create_layerwise_optimizer(model, base_lr=1e-3):
    """
    为不同层设置不同学习率
    
    - 早期层: base_lr * 0.01
    - 中间层: base_lr * 0.1  
    - 后期层: base_lr * 0.5
    - 分类头: base_lr
    """
    param_groups = [
        {'params': [p for n, p in model.named_parameters() 
                   if any(x in n for x in ['conv1', 'bn1', 'layer1', 'layer2'])],
         'lr': base_lr * 0.01, 'name': 'early_layers'},
        {'params': model.layer3.parameters(), 'lr': base_lr * 0.1, 'name': 'layer3'},
        {'params': model.layer4.parameters(), 'lr': base_lr * 0.5, 'name': 'layer4'},
        {'params': model.fc.parameters(), 'lr': base_lr, 'name': 'fc'},
    ]
    return torch.optim.Adam(param_groups)

# 演示
demo_model = models.resnet18(weights='DEFAULT')
demo_model.fc = nn.Linear(512, num_classes)
opt = create_layerwise_optimizer(demo_model)

print("分层学习率配置：")
for g in opt.param_groups:
    print(f"  {g.get('name', 'unknown')}: lr = {g['lr']:.6f}")

---

## 3.4 实用建议清单

### 迁移学习决策流程

```
数据量大？ ──是──→ 域差异大？ ──是──→ 从零训练
    │                  │
   否                 否
    │                  │
    ▼                  ▼
特征提取          微调（解冻部分层）
```

### 关键建议表

| 场景 | 推荐策略 | 学习率 |
|------|----------|--------|
| 数据少 + 域差异小 | 特征提取 | 1e-3 |
| 数据中等 | 微调后几层 | 1e-4 ~ 1e-3 |
| 数据多 + 域差异大 | 微调全部 / 从零训练 | 1e-4 |

### 常见错误

1. ❌ 微调时学习率太大 → 破坏预训练特征
2. ❌ 数据预处理不一致 → 特征不匹配
3. ❌ 只看训练准确率 → 过拟合严重

---

## 📝 本章小结

### 核心知识点

1. **三种迁移策略**：从零训练、特征提取、微调
2. **关键技巧**：ImageNet 预处理、分层学习率
3. **决策因素**：数据量、域差异

### 实验结论

在 Flowers-102 上：迁移学习 > 从零训练，小数据集优势明显。

---

## 🚀 下一步

继续学习 **09_feature_visualization.ipynb** - 特征可视化

In [ ]:
if HAS_TORCH:
    # 训练配置
    def get_optimizer(model, mode='feature_extract'):
        """
        根据迁移学习模式配置优化器
        
        - feature_extract: 只训练 fc 层
        - finetune: 不同层使用不同学习率
        """
        if mode == 'feature_extract':
            # 只优化 fc 层
            return torch.optim.Adam(model.fc.parameters(), lr=1e-3)
        
        elif mode == 'finetune':
            # 不同层使用不同学习率
            # 越靠近输出的层，学习率越高
            param_groups = [
                {'params': model.layer3.parameters(), 'lr': 1e-5},
                {'params': model.layer4.parameters(), 'lr': 1e-4},
                {'params': model.fc.parameters(), 'lr': 1e-3},
            ]
            return torch.optim.Adam(param_groups)
    
    print("优化器配置:")
    print("  特征提取: fc 层 lr=1e-3")
    print("  微调: layer3 lr=1e-5, layer4 lr=1e-4, fc lr=1e-3")
    print("\n💡 越靠近输入的层，学习率越小（保留通用特征）")

---

## 🚀 下一步

继续学习 **09_feature_visualization.ipynb** - 特征可视化